Mange dataframes and trials
------------------



### Environment

In [2]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import pandas as pd
import pickle
import numpy as np
import sys
import os

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [3]:
sys.path.append('/host/projects/sv_experiments/')
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="3"

## Correct wrong paths of old dataset

In [ ]:
voxc1_trial = pd.read_pickle("/dataset/SV_sets/dataframes/voxc1/voxc_trial.pkl")

In [ ]:
voxc1_meta = pd.read_csv("../dataset/voxceleb1/vox1_meta.csv", delimiter="\t")

voxc1_si_df = pd.read_pickle("../dataset/dataframes/voxc1/si_voxc_dataframe.pkl")
voxc1_sv_df = pd.read_pickle("../dataset/dataframes/voxc1/sv_voxc_dataframe.pkl")

voxc12_si_df = pd.read_pickle("../dataset/dataframes/voxc12/si_voxc12_dataframe.pkl")
voxc12_sv_df = pd.read_pickle("../dataset/dataframes/voxc12/sv_voxc12_dataframe.pkl")

In [ ]:
import shutil

for idx, row in voxc12_si_fix_df.iterrows():
    wrong_path = "/dataset/SV_sets/voxceleb2/feats/xvector_npy/" + row.feat
    file_name = '-'.join(wrong_path.rsplit('-', 2)[-2:])
    folder_path = "/dataset/SV_sets/voxceleb2/feats/xvector_npy/" + row.spk
    correct_path = folder_path + '/' + file_name
    if not os.path.isdir(folder_path):
        os.makedirs(folder_path)
    try:
        shutil.move(wrong_path, correct_path)
    except FileNotFoundError:
        print(wrong_path)

In [ ]:
voxc12_si_df.apply(lambda x: x if x.feat )

## voxceleb 1.1 - newly organized voxceleb1

In [ ]:
voxc_11_dev = "/dataset/SV_sets/voxceleb1.1/dev/wav/"
voxc_11_test = "/dataset/SV_sets/voxceleb1.1/test/wav/"

In [ ]:
def append_dir(file):
    return os.path.join(dirpath, file)

In [ ]:
dev_wav_files = []
for dirpath, dirnames, filenames in os.walk(voxc_11_dev):
    if len(filenames) == 0: continue
    dev_wav_files += list(map(append_dir, filenames))

test_wav_files = []
for dirpath, dirnames, filenames in os.walk(voxc_11_test):
    if len(filenames) == 0: continue
    test_wav_files += list(map(append_dir, filenames))

### utt2spk, spk2utt

TODO: sorting order로 저장해야한다!!

In [ ]:
voxc11_df = pd.DataFrame(test_wav_files, columns=['wav'])

In [ ]:
voxc11_df['spk']  = voxc11_df.wav.apply(lambda x: x.split('/')[-3])
voxc11_df['session']  = voxc11_df.wav.apply(lambda x: x.split('/')[-2])
voxc11_df = voxc11_df.set_index(voxc11_df.wav.apply(lambda x: '-'.join(x.split('/')[-3:]).rstrip('.wav')))
voxc11_df.index.name = None
voxc11_df['wav'] = voxc11_df.wav.apply(lambda x: '/'.join(x.split('/')[-3:]))

voxc11_spks = voxc11_df.spk.unique().tolist()

spk2utt_dict = dict()
for spk in voxc11_spks:
    spk2utt_dict[spk] = ' '.join(voxc11_df[voxc11_df.spk == spk].index.tolist())

In [ ]:
with open("/dataset/SV_sets/voxceleb1.1/test/spk2utt", "w") as f:
    for spk, utt in spk2utt_dict.items():
        f.write(spk+" "+utt+"\n")

with open("/dataset/SV_sets/voxceleb1.1/test/utt2spk", "w") as f:
    for idx, row in voxc11_df.iterrows():
        f.write(idx+" "+row.spk+"\n")

In [ ]:
voxc11_df.to_pickle("/dataset/SV_sets/voxceleb1.1/si_voxc11_dataframe.pkl")

### wav.scp

In [ ]:
voxc11_df = pd.DataFrame(test_wav_files, columns=['wav'])
voxc11_df = voxc11_df.set_index(voxc11_df.wav.apply(lambda x: '-'.join(x.split('/')[-3:]).rstrip('.wav')))
voxc11_df.index.name = None

In [ ]:
with open("/dataset/SV_sets/voxceleb1.1/dev/wav.scp", "w") as f:
    for idx, row in voxc11_df.iterrows():
        f.write(idx+' '+row.wav+'\n')

## voxceleb new voxc12

### voxc12 dataframes

In [3]:
voxc12_dev = "/dataset/SV_sets/voxceleb12/feats//"
voxc12_test = "/dataset/SV_sets/voxceleb12/feats/mfcc30_test/"

In [231]:
def append_dir(file):
    return os.path.join(dirpath, file)

In [232]:
dev_npy_files = []
for dirpath, dirnames, filenames in os.walk(voxc12_dev):
    if len(filenames) == 0: continue
    dev_npy_files += list(map(append_dir, filenames))

test_npy_files = []
for dirpath, dirnames, filenames in os.walk(voxc12_test):
    if len(filenames) == 0: continue
    test_npy_files += list(map(append_dir, filenames))

In [233]:
# voxc12_df = pd.DataFrame(dev_npy_files, columns=['wav'])
voxc12_df = pd.DataFrame(test_npy_files, columns=['wav'])

In [234]:
voxc12_df['spk']  = voxc12_df.wav.apply(lambda x: x.split('/')[-2])
voxc12_df = voxc12_df.set_index(voxc12_df.wav.apply(lambda x: '-'.join(x.split('/')[-2:]).rstrip('. npy')))
voxc12_df.index.name = None
voxc12_df['feat'] = voxc12_df.wav.apply(lambda x: '/'.join(x.split('/')[-2:]))
voxc12_df = voxc12_df.drop(columns="wav")

voxc12_spks = voxc12_df.spk.unique().tolist()

In [235]:
voxc12_df['label'] = voxc12_df.groupby('spk').ngroup()

In [237]:
voxc12_df.to_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc12_sv_test_dataframe.pkl")

### voxc1_dev dataframe

In [366]:
voxc12_dev_df = pd.read_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc12_si_train_dataframe.pkl")
voxc12_dev_spks = voxc12_dev_df.spk.unique().tolist()

In [6]:
voxc1_dev_spks = [spk for spk in voxc12_dev_spks if spk.startswith('id1')]
voxc1_dev_df = voxc12_dev_df[voxc12_dev_df.spk.isin(voxc1_dev_spks)]

In [16]:
voxc1_dev_df.loc[:, 'label'] = voxc1_dev_df.groupby('spk').ngroup()

/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [18]:
voxc1_dev_df.to_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc1_si_train_dataframe.pkl")

#### session label


In [35]:
voxc1_dev_df = pd.read_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc1_si_train_dataframe.pkl")

In [48]:
valid_session = voxc1_dev_df.session.value_counts()[voxc1_dev_df.session.value_counts() > 7].index.tolist()

In [49]:
voxc1_dev_df[voxc1_dev_df.session.isin(valid_session)]

,spk,feat,label,session,set,gender
id10001-7w0IBEWc9Qw-00001,id10001,id10001/7w0IBEWc9Qw-00001.npy,0,7w0IBEWc9Qw,train,m
id10001-7w0IBEWc9Qw-00002,id10001,id10001/7w0IBEWc9Qw-00002.npy,0,7w0IBEWc9Qw,train,m
id10001-7w0IBEWc9Qw-00003,id10001,id10001/7w0IBEWc9Qw-00003.npy,0,7w0IBEWc9Qw,train,m
id10001-7w0IBEWc9Qw-00004,id10001,id10001/7w0IBEWc9Qw-00004.npy,0,7w0IBEWc9Qw,train,m
id10001-7w0IBEWc9Qw-00005,id10001,id10001/7w0IBEWc9Qw-00005.npy,0,7w0IBEWc9Qw,train,m
id10001-7w0IBEWc9Qw-00006,id10001,id10001/7w0IBEWc9Qw-00006.npy,0,7w0IBEWc9Qw,train,m
id10001-7w0IBEWc9Qw-00007,id10001,id10001/7w0IBEWc9Qw-00007.npy,0,7w0IBEWc9Qw,train,m
id10001-7w0IBEWc9Qw-00008,id10001,id10001/7w0IBEWc9Qw-00008.npy,0,7w0IBEWc9Qw,train,m
id10001-7w0IBEWc9Qw-00009,id10001,id10001/7w0IBEWc9Qw-00009.npy,0,7w0IBEWc9Qw,train,m
id10001-7w0IBEWc9Qw-00010,id10001,id10001/7w0IBEWc9Qw-00010.npy,0,7w0IBEWc9Qw,train,m


### speaker identificaiton data split 

In [169]:
voxc12_dev_df['session'] = voxc12_dev_df.index

In [170]:
voxc12_dev_df['session'] = voxc12_dev_df.session.apply(lambda x: x[8:-6])

In [179]:
voxc12_dev_test_sessions = voxc12_dev_df.groupby('spk').apply(lambda x: x.session.sample(n=1))

In [190]:
voxc12_dev_df['set'] = 'train'

In [198]:
voxc12_dev_df.loc[voxc12_dev_df.session.isin(voxc12_dev_test_sessions), 'set'] = 'val'

In [200]:
len(voxc12_dev_df[voxc12_dev_df.set == 'val']) / len(voxc12_dev_df)

0.0958903787859809

In [201]:
voxc12_dev_df.to_pickle('/dataset/SV_sets/voxceleb12/dataframes/voxc12_si_train_dataframe.pkl')

### gender split

In [480]:
voxc1_dev_df = pd.read_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc1_si_train_dataframe.pkl")
voxc1_meta = pd.read_csv("/dataset/SV_sets/voxceleb12/vox1_meta.csv")
voxc1_meta = voxc1_meta.set_index('VoxCeleb1 ID')
voxc1_dev_df['gender'] = voxc1_dev_df.spk.apply(lambda x: voxc1_meta.loc[x].Gender)
voxc1_dev_df.to_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc1_si_train_dataframe.pkl")

In [484]:
voxc1_test_df = pd.read_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc12_sv_test_dataframe.pkl")
voxc1_meta = pd.read_csv("/dataset/SV_sets/voxceleb12/vox1_meta.csv")
voxc1_meta = voxc1_meta.set_index('VoxCeleb1 ID')
voxc1_test_df['gender'] = voxc1_test_df.spk.apply(lambda x: voxc1_meta.loc[x].Gender)
voxc1_test_df.to_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc12_sv_test_dataframe.pkl")

In [486]:
voxc1_dev_m = voxc1_dev_df[voxc1_dev_df.gender == 'm']
voxc1_dev_f = voxc1_dev_df[voxc1_dev_df.gender == 'f']

In [499]:
voxc1_dev_m['label'] = voxc1_dev_m.groupby('spk').ngroup()
voxc1_dev_f['label'] = voxc1_dev_f.groupby('spk').ngroup()

/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [500]:
voxc1_dev_m.to_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc1_m_si_train_dataframe.pkl")
voxc1_dev_f.to_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc1_f_si_train_dataframe.pkl")

In [502]:
voxc1_meta.to_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc1_meta.pkl")

### trial

여러 trial이 있지만 일단은 voxc1의 sv_set을 사용하는 trial을 공용 trial로 지정했다.

In [15]:
voxc1_test_df = pd.read_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc12_sv_test_dataframe.pkl") # id의 기준이 되는 df
voxc1_test_spks = voxc1_test_df.spk.unique().tolist()

In [16]:
voxc1_trial = pd.read_csv("/dataset/SV_sets/voxceleb12/trial_data/veri_test.txt", sep=' ', header=None)
voxc1_trial.columns = ['label', 'enroll', 'test']

In [17]:
enroll_idx = voxc1_trial.enroll.apply(lambda x: x.replace('/', '-').rstrip('.wav'))
test_idx = voxc1_trial.test.apply(lambda x: x.replace('/', '-').rstrip('.wav'))

voxc1_trial['enroll_idx'] = enroll_idx
voxc1_trial['test_idx'] = test_idx

In [18]:
enroll_num_idx = voxc1_trial.enroll_idx.apply(lambda x: voxc1_test_df.index.get_loc(x))
test_num_idx = voxc1_trial.test_idx.apply(lambda x: voxc1_test_df.index.get_loc(x))

voxc1_trial['enrolment_id'] = enroll_num_idx
voxc1_trial['test_id'] = test_num_idx

In [19]:
voxc1_trial = voxc1_trial.drop(columns=['enroll', 'test'])

In [20]:
voxc1_trial.to_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc12_test_trial.pkl")

plda를 위한 trial file

In [26]:
voxc1_trial['is_target'] = voxc1_trial.label.apply(lambda x: 'target' if x==1 else 'nontarget')

In [28]:
voxc1_trial.to_csv("/dataset/SV_sets/voxceleb12/trials/voxceleb12_sv", sep=' ', header=False, index=False, columns=['enroll_idx', 'test_idx', 'is_target'])

#### female-female case를 많이 추가한 trial

In [583]:
voxc1_m_spks = voxc1_meta[voxc1_meta.Gender == 'm'].index.tolist()

In [584]:
voxc1_f_spks = voxc1_meta[voxc1_meta.Gender == 'f'].index.tolist()

In [587]:
voxc1_test_m_spks = [spk for spk in voxc1_test_spks if spk in voxc1_m_spks]

In [589]:
voxc1_test_f_spks = [spk for spk in voxc1_test_spks if spk in voxc1_f_spks]

In [557]:
# positive pair
import itertools
pos_trials = []
pos_len = dict()
for spk in voxc1_test_spks:
    utters = voxc1_test_df[voxc1_test_df.spk == spk].index
    every_pos_pair = list(filter(diff_session, itertools.combinations(utters, 2)))
    pos_trials += every_pos_pair
    pos_len[spk] = len(every_pos_pair) 

In [560]:
def diff_session(pair):
    a, b = pair
    if a[8:19] != b[8:19]:
        return True
    else:
        return False

In [610]:
# negative m pair
import itertools
import random
neg_m_trials = []
for spk in voxc1_test_m_spks:
    utters = voxc1_test_df[ (voxc1_test_df.spk == spk)].index.tolist()
    neg_targets = voxc1_test_df[(voxc1_test_df.gender == 'm') & (voxc1_test_df.spk != spk)].index.tolist()
    total_pairs = list(itertools.product(utters, neg_targets))
    neg_m_trials += list(filter(diff_session, total_pairs))

In [611]:
# negative f pair
import itertools
import random
neg_f_trials = []
for spk in voxc1_test_f_spks:
    utters = voxc1_test_df[ (voxc1_test_df.spk == spk)].index.tolist()
    neg_targets = voxc1_test_df[(voxc1_test_df.gender == 'f') & (voxc1_test_df.spk != spk)].index.tolist()
    total_pairs = list(itertools.product(utters, neg_targets))
    neg_f_trials += list(filter(diff_session, total_pairs))

In [626]:
pos_trials = random.choices(pos_trials, k=10000)

In [627]:
neg_m_trials = random.choices(neg_m_trials, k=int(len(pos_trials)/2))

In [628]:
neg_f_trials = random.choices(neg_f_trials, k=int(len(pos_trials)/2))

In [629]:
neg_trials = neg_m_trials + neg_f_trials

In [630]:
voxc1_trials = pos_trials + neg_trials
voxc1_trial_df = pd.DataFrame(voxc1_trials, columns=["enroll_idx", "test_idx"])

In [631]:
voxc1_trial_df['enroll_uttr_id'] = voxc1_trial_df.enroll_idx.apply(lambda x: voxc1_test_df.index.get_loc(x))
voxc1_trial_df['test_uttr_id'] = voxc1_trial_df.test_idx.apply(lambda x: voxc1_test_df.index.get_loc(x))
voxc1_trial_df['enroll_spk'] = voxc1_trial_df.enroll_idx.apply(lambda x: x[:7])
voxc1_trial_df['test_spk'] = voxc1_trial_df.test_idx.apply(lambda x: x[:7])
voxc1_trial_df['label'] = [1]*len(pos_trials) + [0]*len(neg_trials)
voxc1_trial_df.to_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc12_bal_sv_test_dataframe.pkl")

In [652]:
voxc1_trial_df['is_target'] = voxc1_trial_df.label.apply(lambda x: 'target' if x==1 else 'nontarget')
voxc1_trial_df.to_csv("/dataset/SV_sets/voxceleb12/trials/voxceleb12_bal_sv", sep=' ', header=False, index=False, columns=['enroll_idx', 'test_idx', 'is_target'])

## voxceleb2

In [661]:
#voxc2 trial 기준 dataframe
voxc2_sv_test_df = pd.read_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc2_sv_test_dataframe.pkl")

### trials

In [674]:
voxc2_all_trial_df = pd.read_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc2_trials/voxc2_all_trials.pkl")
voxc2_all_trial_df['enroll_spk']     = voxc2_all_trial_df.enroll_idx.apply(lambda x: x[:7])
voxc2_all_trial_df['test_spk']       = voxc2_all_trial_df.test_idx.apply(lambda x: x[:7])
voxc2_all_trial_df.to_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc2_trials/voxc2_all_trials.pkl")

In [682]:
voxc2_hard_trial_df = pd.read_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc2_trials/voxc2_hard_trial.pkl")
voxc2_hard_trial_df['enroll_spk']     = voxc2_all_trial_df.enroll_idx.apply(lambda x: x[:7])
voxc2_hard_trial_df['test_spk']       = voxc2_all_trial_df.test_idx.apply(lambda x: x[:7])
voxc2_hard_trial_df.to_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc2_trials/voxc2_hard_trials.pkl")

In [684]:
voxc2_all_trial_df['is_target'] = voxc2_all_trial_df.label.apply(lambda x: 'target' if x==1 else 'nontarget')
voxc2_all_trial_df.to_csv("/dataset/SV_sets/voxceleb12/kaldi_files/voxc2_all_sv", sep=' ', header=False, index=False, columns=['enroll_idx', 'test_idx', 'is_target'])

In [685]:
voxc2_hard_trial_df['is_target'] = voxc2_hard_trial_df.label.apply(lambda x: 'target' if x==1 else 'nontarget')
voxc2_hard_trial_df.to_csv("/dataset/SV_sets/voxceleb12/kaldi_files/voxc2_hard_sv", sep=' ', header=False, index=False, columns=['enroll_idx', 'test_idx', 'is_target'])

### utt2spk spk2utt

In [294]:
spk2utt = voxc2_si_train_df.sort_values("spk").groupby("spk").apply(lambda x: x.uttr_id.values)

In [299]:
with open("/dataset/SV_sets/gcommand/kaldi_files/spk2utt", "w") as f:
    for k, v in spk2utt.iteritems():
        line = ' '.join([k]+v.tolist())
        f.write(line+'\n')

In [300]:
utt2spk = all_gcommand_df.sort_values("uttr_id")['spk']

In [301]:
with open("/dataset/SV_sets/gcommand/kaldi_files/utt2spk", "w") as f:
    for k, v in utt2spk.iteritems():
        line = ' '.join([k, v])
        f.write(line+'\n')

## gcommand 

In [4]:
gcommand_train_df = pd.read_pickle("/dataset/SV_sets/gcommand/dataframes/equal_num_30spk/equal_num_30spk_si.pkl")
gcommand_test_df = pd.read_pickle("/dataset/SV_sets/gcommand/dataframes/equal_num_30spk/equal_num_30spk_sv.pkl")

In [19]:
val_uttrs_idx = gcommand_train_df.groupby('spk', group_keys=False).apply(lambda x: x.sample(n=2)).index

In [20]:
gcommand_train_df.set = 'train'

In [32]:
gcommand_train_df.label

004ae714-down-0        0
004ae714-eight-0       0
004ae714-five-0        0
004ae714-four-0        0
004ae714-four-1        0
004ae714-go-0          0
004ae714-go-1          0
004ae714-left-0        0
004ae714-off-0         0
004ae714-off-1         0
004ae714-on-0          0
004ae714-on-1          0
004ae714-one-0         0
004ae714-seven-0       0
004ae714-seven-1       0
004ae714-seven-2       0
004ae714-six-0         0
004ae714-six-1         0
004ae714-stop-0        0
004ae714-up-0          0
004ae714-yes-0         0
004ae714-yes-1         0
004ae714-zero-0        0
004ae714-zero-1        0
00b01445-down-0        1
00b01445-down-1        1
00b01445-eight-0       1
00b01445-five-0        1
00b01445-five-1        1
00b01445-go-0          1
                    ... 
ffd2ba2f-up-4       1757
ffd2ba2f-wow-0      1757
ffd2ba2f-yes-0      1757
ffd2ba2f-yes-1      1757
ffd2ba2f-yes-2      1757
ffd2ba2f-yes-3      1757
ffd2ba2f-yes-4      1757
ffd2ba2f-zero-0     1757
ffd2ba2f-zero-1     1757


In [29]:
gcommand_train_df.loc[val_uttrs_idx, 'set'] = 'val'

In [30]:
gcommand_train_df.set.value_counts()

train    41380
val       3518
Name: set, dtype: int64

In [31]:
gcommand_train_df.to_pickle("/dataset/SV_sets/gcommand/dataframes/equal_num_30spk/equal_num_30spk_si.pkl")

In [458]:
all_gcommands_df = pd.read_pickle("/dataset/SV_sets/gcommand/dataframes/all_gcommands.pkl")

### utt2spk spk2utt

In [459]:
spk2utt = gcommand_train_df.sort_values("spk").groupby("spk").apply(lambda x: x.uttr_id.values)

In [460]:
with open("/dataset/SV_sets/gcommand/kaldi_files/train_spk2utt", "w") as f:
    for k, v in spk2utt.iteritems():
        line = ' '.join([k]+v.tolist())
        f.write(line+'\n')

In [461]:
utt2spk = gcommand_train_df.sort_values("uttr_id")['spk']

In [462]:
with open("/dataset/SV_sets/gcommand/kaldi_files/train_utt2spk", "w") as f:
    for k, v in utt2spk.iteritems():
        line = ' '.join([k, v])
        f.write(line+'\n')

### trials (self-made)

In [399]:
gcommand_test_spks = gcommand_test_df.spk.unique().tolist()

In [438]:
# positive pair
import itertools
pos_trials = []
pos_len = []
for spk in gcommand_test_spks:
    utters = gcommand_test_df[gcommand_test_df.spk == spk].index
    every_pos_pair = itertools.combinations(utters, 2)
    pos_trials += every_pos_pair

In [439]:
# positive pair
import itertools
import random
neg_trials = []
for spk in gcommand_test_spks:
    utters = gcommand_test_df[gcommand_test_df.spk == spk].index.tolist()
    neg_targets = gcommand_test_df[gcommand_test_df.spk != spk].index.tolist()
    total_pairs = list(itertools.product(utters, neg_targets))
    neg_trials += random.choices(total_pairs, k=5995)

In [441]:
gcommand_trials = pos_trials + neg_trials
gcommand_trial_df = pd.DataFrame(gcommand_trials, columns=["enroll_idx", "test_idx"])

In [448]:
gcommand_trial_df['enrolment_id'] = gcommand_trial_df.enroll_idx.apply(lambda x: gcommand_test_df.index.get_loc(x))

gcommand_trial_df['test_id'] = gcommand_trial_df.test_idx.apply(lambda x: gcommand_test_df.index.get_loc(x))

gcommand_trial_df['enroll_spk'] = gcommand_trial_df.enroll_idx.apply(lambda x: x[:8])

gcommand_trial_df['test_spk'] = gcommand_trial_df.test_idx.apply(lambda x: x[:8])

gcommand_trial_df['label'] = [1]*len(pos_trials) + [0]*len(neg_trials)

gcommand_trial_df.to_pickle("/dataset/SV_sets/gcommand/dataframes/equal_num_30spk/equal_num_30spk_test_trial.pkl")

In [450]:
gcommand_trial_df['is_target'] = gcommand_trial_df.label.apply(lambda x: 'target' if x==1 else 'nontarget')

gcommand_trial_df.to_csv("/dataset/SV_sets/gcommand/kaldi_files/gcommand_30spks_sv", sep=' ', header=False, index=False, columns=['enroll_idx', 'test_idx', 'is_target'])

In [451]:
gcommand_train_df = gcommand_train_df.sort_index()

In [452]:
gcommand_train_df['label'] = gcommand_train_df.groupby('spk').ngroup()

In [453]:
gcommand_train_df.to_pickle("/dataset/SV_sets/gcommand/dataframes/equal_num_30spk/equal_num_30spk_si.pkl")